In [61]:
import nibabel as nib
import numpy as np
import requests
import os
import re

In [62]:
def remove_invalid_chars(filename, replacement=''):
     
    invalid_chars_pattern = r'[\/:*?"<>|]'
    
    # Replace invalid characters with the specified replacement character
    new_filename = re.sub(invalid_chars_pattern, replacement, filename)
    
    return new_filename

In [63]:
def fetch_study_as_nifti(server_url, study_orthanc_id):
    
    # DICOM Web URL for fetching study
    study_url = f"{server_url}/studies/{study_orthanc_id}"

    # Send GET request to fetch series
    response = requests.get(study_url)
    
    # Check if request was successful
    if response.status_code == 200:
        study_data = response.json()
                
        series_count = len(study_data["Series"])

        print(f"Found {series_count} series in study {study_orthanc_id}")
        
        # Create directory to save DICOM files
        study_dir = study_data["MainDicomTags"]["StudyInstanceUID"]
        os.makedirs(str(study_dir), exist_ok=True)
                
        # Iterate through series and fetch instances
        for series_orthanc_id in study_data["Series"]:
            print(f"Fetching series {series_orthanc_id}")
            fetch_nifti_series_from_orthanc(server_url, series_orthanc_id, study_dir)
    else:
        print("Failed to fetch study.")
        print(f"Status Code: {response.status_code}")
        print(response.text)

In [64]:
def fetch_nifti_series_from_orthanc(server_url, series_orthanc_id, save_dir):
    
    series_url = f"{server_url}/series/{series_orthanc_id}"
    # Send GET request to fetch DICOM instance
    data = requests.get(series_url)
    response = requests.get(series_url + "/nifti?compress")
    
    series_description = data.json()["MainDicomTags"]["SeriesDescription"] if "SeriesDescription" in data.json()["MainDicomTags"] else "unknown"
    series_description = remove_invalid_chars(series_description)

    # Check if request was successful
    if response.status_code == 200:
        # Save DICOM file
        with open(os.path.join(save_dir, str(series_description + ".nii.gz")), 'wb') as f:
            f.write(response.content)
        print(f"Saved nifti series {series_orthanc_id}")
    else:
        print("Failed to fetch DICOM instance.")
        print(f"Status Code: {response.status_code}")
        print(response.text)

In [65]:
server_url = "http://localhost:8042"
study = "2c9a0594-c963ae88-94ddbbcf-88b1bb47-be7d812b"

fetch_study_as_nifti(server_url, study)

Found 6 series in study 2c9a0594-c963ae88-94ddbbcf-88b1bb47-be7d812b
Fetching series 7e4e2a42-4aa60232-dc740434-a1702956-852fd45c


Saved nifti series 7e4e2a42-4aa60232-dc740434-a1702956-852fd45c
Fetching series 221fb4a3-1c708d50-b99c15cb-a76e5611-0e0d7822
Saved nifti series 221fb4a3-1c708d50-b99c15cb-a76e5611-0e0d7822
Fetching series d3acbdfd-09cb4e8f-6bba59cf-ca28a8f5-430e73dd
Saved nifti series d3acbdfd-09cb4e8f-6bba59cf-ca28a8f5-430e73dd
Fetching series 076467ec-fec949b4-8046d2a6-f497b296-e2233406
Saved nifti series 076467ec-fec949b4-8046d2a6-f497b296-e2233406
Fetching series c89e8b65-cda87f20-633bef75-9c147dd3-e15b5b54
Saved nifti series c89e8b65-cda87f20-633bef75-9c147dd3-e15b5b54
Fetching series f33edafe-49c2417e-93c83283-9bddc4df-35d13074
Saved nifti series f33edafe-49c2417e-93c83283-9bddc4df-35d13074
